In [ ]:
# Initialize seed
from emobpy.tools import set_seed
set_seed()

------------------------------------------------
## Step 1: Vehicle mobility time series
------------------------------------------------
a) generation of a time series

------------------------------------------------

In [ ]:
from emobpy import Mobility

In [ ]:
m = Mobility(config_folder='config_files')

In [ ]:
m.set_params(
             name_prefix="BEV1",
             total_hours=168, # one week
             time_step_in_hrs=0.25, # 15 minutes
             category="user_defined",
             reference_date="01/01/2020"
            )

In [ ]:
m.set_stats(
            stat_ntrip_path="TripsPerDay.csv",
            stat_dest_path="DepartureDestinationTrip.csv",
            stat_km_duration_path="DistanceDurationTrip.csv",
            )

In [ ]:
m.set_rules(rule_key="user_defined") # see /config_files/rules.yml, it contains a dictionary
                                     # whose key must be the same as rule_key. 
                                     # To see all possible rules `from emobpy.constants import RULE; RULE` 

In [ ]:
m.run()

In [ ]:
m.save_profile(folder="db", description='168 hrs 15 min step ref-date 01/01/2020')

-------------------------
b) See some attributes

-------------------------

In [ ]:
m.name

In [ ]:
m.kind

In [ ]:
m.description

In [ ]:
m.timeseries

In [ ]:
m.profile.head()

-------------------------------
c) Profiles management

-------------------------------

In [ ]:
from emobpy import DataBase

In [ ]:
DB = DataBase('db')

In [ ]:
DB.loadfiles()

In [ ]:
DB.db.keys()

In [ ]:
DB.db[m.name]['user_rules']

In [ ]:
DB.db[m.name]['kind']

-------------------------
d) Visualization

-------------------------

In [ ]:
from emobpy.plot import NBplot

In [ ]:
PLT = NBplot(DB)

In [ ]:
fig_mobility = PLT.sgplot_dp(m.name)

In [ ]:
fig_mobility

------------------------------------------------------
## Step 2: Driving consumption time series
------------------------------------------------------
a) Vehicle model configuration

------------------------------------------------------

In [ ]:
from emobpy import Consumption, HeatInsulation, BEVspecs

In [ ]:
DB.update()                # This load new files hosted in database folder as result of new generated files

In [ ]:
# mname = list(DB.db.keys())[0]      # getting the id of the first mobility profile
mname = m.name

In [ ]:
HI = HeatInsulation(True)            # Creating the heat insulation by copying the default configuration

In [ ]:
BEVS = BEVspecs()                    # Database that contains BEV models

In [ ]:
BEVS.show_models()

In [ ]:
BEVS.parameters

In [ ]:
dataframe = BEVS.search_by_parameter()

In [ ]:
VW_ID3 = BEVS.model(('Volkswagen','ID.3',2020))    # Model instance that contains vehicle parameters

In [ ]:
VW_ID3.parameters

----------------------------------------------------------------------
b) Calculate consumption for each trip and generate the time series

----------------------------------------------------------------------

In [ ]:
c = Consumption(mname, VW_ID3)

In [ ]:
c.load_setting_mobility(DB)

In [ ]:
c.run(
    heat_insulation=HI,
    weather_country='DE',
    weather_year=2016,
    passenger_mass=75,                   # kg
    passenger_sensible_heat=70,          # W
    passenger_nr=1.5,                    # Passengers per vehicle including driver
    air_cabin_heat_transfer_coef=20,     # W/(m2K). Interior walls
    air_flow = 0.02,                     # m3/s. Ventilation
    driving_cycle_type='WLTC',           # Two options "WLTC" or "EPA"
    road_type=0,                         # For rolling resistance, Zero represents a new road.
    road_slope=0
    )

In [ ]:
c.save_profile('db')

------------------------------
c) See some attributes

------------------------------

In [ ]:
c.name

In [ ]:
c.kind

In [ ]:
c.input

In [ ]:
c.brand, c.model, c.year

In [ ]:
c.timeseries # Consumption in kWh/timestep -> timestep 15 min in this example

In [ ]:
c.profile.head()

-------------------------
d) Visualization

-------------------------

In [ ]:
fig_consumption = PLT.sankey(c.name, include=None, to_html=False, path=None)

In [ ]:
fig_consumption

-------------------------------------------------------------------
## Step 3: Grid availability time series
-------------------------------------------------------------------
a) probability distribution and power rating for charging stations

-------------------------------------------------------------------

In [ ]:
from emobpy import Availability

In [ ]:
DB.update()                               # This load new generated files that are hosted in database folder
cname = c.name                            # getting the id of the first consumption profile


station_distribution = {                  # Dictionary with charging stations type probability distribution per the purpose of the trip (location or destination)
    'prob_charging_point': {
        'errands': {'public': 0.5, 'none': 0.5},
        'escort': {'public': 0.5, 'none': 0.5},
        'leisure': {'public': 0.5, 'none': 0.5},
        'shopping': {'public': 0.5, 'none': 0.5},
        'home': {'public': 0.5, 'none': 0.5},
        'workplace': {'public': 0.0, 'workplace': 1.0, 'none': 0.0},   # If the vehicle is at the workplace, it will always find a charging station available (assumption)
        'driving': {'none': 0.99, 'fast75': 0.005, 'fast150': 0.005}}, # with the low probability given to fast charging is to ensure fast charging only for very long trips (assumption)
    'capacity_charging_point': {                                       # Nominal power rating of charging station in kW
        'public': 22,
        'home': 3.7,
        'workplace': 11,
        'none': 0,  # dummy station
        'fast75': 75,
        'fast150': 150}
}

--------------------------------
b) Generate time series

--------------------------------

In [ ]:
ga = Availability(cname, DB)

In [ ]:
ga.set_scenario(station_distribution)

In [ ]:
ga.run()

In [ ]:
ga.save_profile('db')

--------------------------------
b) See some attributes

--------------------------------

In [ ]:
ga.name

In [ ]:
ga.input

In [ ]:
ga.kind

In [ ]:
ga.battery_capacity

In [ ]:
ga.charging_eff

In [ ]:
ga.discharging_eff

In [ ]:
ga.soc_init

In [ ]:
ga.soc_min

In [ ]:
ga.timeseries

-------------------------
c) Visualization

-------------------------

In [ ]:
fig_availability = PLT.sgplot_ga(ga.name, rng=None, to_html=False, path=None)

In [ ]:
fig_availability

----------------------------------------------------------
## Step 4: Grid electricity demand time series
----------------------------------------------------------
a) Selection of charging strategies

---------------------------------------

In [ ]:
from emobpy import Charging

In [ ]:
DB.update()

aname = ga.name                            # getting the id of the availability profile

strategies = [
              "immediate",                 # When battery has SOC < 100% then it charges immediatelly at a maximun power rating of the current charging station
              "balanced",                  # When battery has SOC < 100% then it charges immediatelly but at lower rating power to ensure 100% SOC at the end (before moving to another place).
              "from_0_to_24_at_home",      # Customized: starting time of charging (this case 0 hrs), final time of charging (this case 24 hrs), at could be one 'location' (this case 'home') or 'any'.
              "from_23_to_8_at_any"
             ]

------------------------------------------------------
b) generation of 4 grid electricity demand time series

------------------------------------------------------

In [ ]:
for option in strategies:
    ged = Charging(aname)
    ged.load_scenario(DB)
    ged.set_sub_scenario(option)
    ged.run()
    print(f'Creation Successful:{ged.success}')   # if False, modify the strategy to a less constrained.
    ged.save_profile('db')

-----------------------------------------------------------------------------
c) See some atributes for the last created time series as an example

-----------------------------------------------------------------------------

In [ ]:
ged.name

In [ ]:
ged.input

In [ ]:
ged.kind

In [ ]:
ged.option

In [ ]:
ged.timeseries

In [ ]:
ged.profile

-------------------------
d) Visualization

-------------------------

In [ ]:
fig_ged = PLT.sgplot_ged(ged.name, rng=None, to_html=False, path=None) # this looks for all strategies of a single grid availability time series, even though we provide one grid demand id

In [ ]:
fig_ged

------------------------------------------------------
### Visualize all time series of a vehicle profile
------------------------------------------------------

In [ ]:
fig_channel = PLT.overview(ged.name)

In [ ]:
fig_channel

------------------------------------------------------------
### Export all time-series in 'db' folder to [DIETER](https://diw-evu.gitlab.io/dieter_public/dieterpy/) format
-----------------------------------------------------------------------------------------------------------
After exporting see the two CSV files at "db" folder

--------------------------------------------------------------------

In [ ]:
from emobpy import Export

In [ ]:
DB.update()
Exp = Export()
Exp.loaddata(DB)
Exp.to_csv()
Exp.save_files()

-------------------------------------------------------------------------------
### Download weather data from [Zenodo](https://zenodo.org/record/1489915)
-------------------------------------------------------------------------------
Total size: 300 MB

This function allows us to select different countries and years when creating new driving consumption time-series.

After finishing the download, you will get the location of the files on your PC. If you want to add more countries, you can edit the CSV files there.

emobpy includes Germany's weather data only for 2016. This data set has several European countries and years from 2000-2017.

-------------------------------------------------------------------------------

In [ ]:
# from emobpy import Weather

In [ ]:
# WD = Weather()

In [ ]:
# WD.download_weather_data()